<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_otto/blob/main/otto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle --upgrade

In [1]:
import os

os.environ['KAGGLE_USERNAME'] = 'mmmmmlee'
os.environ['KAGGLE_KEY'] = 'b4527f98e27813c6dae3fb199096dcb5'

In [ ]:
!kaggle -h
# 잘 설치 되었는지 확인하기

In [2]:
!kaggle competitions download -c otto-recommender-system
# 캐글 데이터 갖고오기

 99% 1.92G/1.94G [00:13<00:00, 148MB/s]
100% 1.94G/1.94G [00:13<00:00, 154MB/s]


In [3]:
!unzip 'otto-recommender-system.zip'
# 압축 풀어주기

Archive:  otto-recommender-system.zip
  inflating: sample_submission.csv   
  inflating: test.jsonl              
  inflating: train.jsonl             


In [ ]:
!pip install jsonlines

In [6]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 63.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import json
import csv
import polars as pl
from io import StringIO
# import jsonlines


In [3]:
with open ('test.jsonl', encoding='utf-8') as file:
  test = [json.loads(line) for line in file]

In [ ]:
test = pd.DataFrame(test)

In [ ]:
# data = []
# with open('test.jsonl', encoding='utf-8') as f:
#   for line in f:
#     data.append(line)

In [ ]:
# with open ('train.jsonl', encoding='utf-8') as file:
#   train = [json.loads(line) for line in file]
#   train = pd.DataFrame(train)
#   train.head()
# 용량이 커서 RAM 오류남

In [ ]:
# with jsonlines.open('test.jsonl') as f:
#   for line in f.iter():
#     print(line['aid'])
#     print(line['ts'])
#     print(line['type'])

In [5]:
with open('test.jsonl', encoding='utf-8') as file:
    test = file.readlines()
# train 셋 가져오기

In [6]:
slice_size = 1_000_000
df = pl.concat(
    [
        pl.read_json(
            StringIO("".join(test[offset: (offset + slice_size)])),
            json_lines=True,
        )
        for offset in range(0, len(test), slice_size)
    ]
)
#converting struct of events in 3 columns(aid,ts,type) 
test=df.explode('events').unnest('events')
# event 쪼개주기

In [7]:
test.head()

session,aid,ts,type
i64,i64,i64,str
12899779,59625,1661724000278,"""clicks"""
12899780,1142000,1661724000378,"""clicks"""
12899780,582732,1661724058352,"""clicks"""
12899780,973453,1661724109199,"""clicks"""
12899780,736515,1661724136868,"""clicks"""


In [3]:
train_1 = pd.DataFrame()
chunks = pd.read_json('train.jsonl', lines=True, chunksize=100_000)

for e, chunk in enumerate(chunks):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    if e < 2:
        # train_1 = pd.concat([train_sessions, chunk])
        for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
            for event in events:
                event_dict['session'].append(session)
                event_dict['aid'].append(event['aid'])
                event_dict['ts'].append(event['ts'])
                event_dict['type'].append(event['type'])
        chunk_session = pd.DataFrame(event_dict)
        train_1 = pd.concat([train_1, chunk_session])
    else:
        break
        
train_1 = train_1.reset_index(drop=True)

In [4]:
train_1

,session,aid,ts,type
0,0,1517085,1659304800025,clicks
1,0,1563459,1659304904511,clicks
2,0,1309446,1659367439426,clicks
3,0,16246,1659367719997,clicks
4,0,1781822,1659367871344,clicks
...,...,...,...,...
10285556,199999,928064,1659336408967,clicks
10285557,199999,849970,1659336449078,clicks
10285558,199999,1052480,1659336547035,clicks
10285559,199999,487255,1659336561116,clicks
